## 유튜브의 자막을 추출하고 그에 따라 내용을 요약

In [19]:
!pip install youtube-transcript-api


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [25]:
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

video_id='orDKvo8h71o'
captions = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
texts = [caption['text'] for caption in captions]

# TF-IDF를 사용하여 각 자막의 중요도 계산
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
importance_scores = np.mean(tfidf_matrix, axis=1).A1

# 중요 지점을 감지하기 위한 임계값 설정 (예: 상위 20% 지점을 중요 지점으로 설정)
threshold = np.percentile(importance_scores, 80)

# 중요 지점 추출
important_points = []
for idx, score in enumerate(importance_scores):
    if score >= threshold:
        important_points.append((captions[idx]['start'], captions[idx]['start'] + captions[idx]['duration']))

In [40]:
# 중요 지점의 자막을 추출하고 요약
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()
summaries = []

for start, end in important_points:
    # 해당 시간 동안의 자막 추출
    segment_texts = [caption['text'] for caption in captions if start <= caption['start'] < end]
    segment_text = ' '.join(segment_texts)
    # GPT-4를 이용해 요약
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature=0.5,
        messages=[
            {
                "role": "system",
                "content": "다음과 같이 특정 구간의 자막: {segment_text}을 제시하면 당신은 한글로 내용을 요약해 주고 필요하다면 설명도 추가해줘.",
            },
            {
                "role": "user",
                "content": f"자막: {segment_text}",
            },
        ],
    )

    summary = response.choices[0].message
    summaries.append({'start': start, 'end': end, 'summary': summary})

# 결과 출력
for summary in summaries:
    print(f"중요 지점: {summary['start']}초 ~ {summary['end']}초")
    print(f"요약: {summary['summary'].content}\n")

중요 지점: 5.04초 ~ 10.36초
요약: ChatCompletionMessage(content='Kangan is currently working as a research scientist.', role='assistant', function_call=None, tool_calls=None)

중요 지점: 10.36초 ~ 15.079999999999998초
요약: ChatCompletionMessage(content='고객은 OpenAI의 GBD 팀에 속해 있고 대규모 프로젝트의 다양한 측면에 참여하고 있다고 말씀하셨습니다.', role='assistant', function_call=None, tool_calls=None)

중요 지점: 38.68초 ~ 44.079초
요약: ChatCompletionMessage(content='방금 MIT에서 박사 학위를 받았습니다. 축하해주세요!', role='assistant', function_call=None, tool_calls=None)

중요 지점: 54.48초 ~ 59.44초
요약: ChatCompletionMessage(content='사용자가 "안녕하세요, 제 이름은 안이고 오늘 여기 있어서 정말 행복합니다"라고 말했습니다.', role='assistant', function_call=None, tool_calls=None)

중요 지점: 76.36초 ~ 80.079초
요약: ChatCompletionMessage(content='AI의 미래를 형성하는 데 참여할 수 있다는 생각이 드는데, 이에 대해 이야기해볼까요.', role='assistant', function_call=None, tool_calls=None)

중요 지점: 81.88초 ~ 86.64초
요약: ChatCompletionMessage(content='"우리는 정말로 올바르게 해야 하므로 고려할만한 주제일 것 같습니다."', role='assistant', function_call=None, tool_calls=None)

중요 지